In [ ]:
import os
import sys
import librosa
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
SAMPLE_RATE = 22050
DURATION = 262144/SAMPLE_RATE  # in seconds

In [ ]:
#run for GPU
pyhsical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(pyhtiscal_devices[0],True)

In [ ]:
# Load data not in memory
dir = "C:/Users/den/OneDrive - ORDIX AG/OneDrive/10_Studium/Masterarbeit/Python/testing"
file_name = "waveform_data.npy"


file_path = os.path.join(dir, file_name)
ar = np.load(file_path,mmap_mode="r",allow_pickle=True)

ar[0].shape
#np.array(ar[0])

# Load Data

In [ ]:
FILES_DIR = "C:/Masterarbeit/music"
FILES_DIR_SAVE = "C:/Masterarbeit/music_npy/sr_22050"
FILENAME_DIR = "C:/Masterarbeit"
MONO = True   

In [ ]:
signal = librosa.load(os.path.join(FILES_DIR, "accidentally - onoyoko.mp3"),
                      sr=5512,
                      #duration=30,
                      mono=True)[0]

In [ ]:
def Loader(file_path, sample_rate, duration, mono):

    signal = librosa.load(file_path,
                          sr=sample_rate,
                          # duration=duration,
                          mono=mono)[0]
    return signal


def split_signal(signal):
    i = 0
    j = 0
    amount_samples_in_segment = DURATION * SAMPLE_RATE
    amount_splits = (len(signal))//amount_samples_in_segment
    splitted_signal = []
    for _ in range(amount_splits):
        i = i + amount_samples_in_segment
        if len(signal[j:i]) == amount_samples_in_segment:
            splitted_signal.append(signal[j:i])
        j = j + amount_samples_in_segment
    return np.array(splitted_signal)


signals = []
for root, _, files in os.walk(FILES_DIR):
    for file in files:
        file_path = os.path.join(root, file)
        signal = Loader(file_path, SAMPLE_RATE, DURATION, MONO)
        splitted_signal = split_signal(signal)
        i = 0
        for signal in splitted_signal:
            i = i + 1
            filename = os.path.join(FILES_DIR_SAVE, file)
            np.save(f"{filename}_{i}",np.array(signal))


In [ ]:
# create file with all the filenames
filenames=[]
for root,_,files in os.walk(FILES_DIR_SAVE):
    for file in files:
        filenames.append(file)
filename = os.path.join("C:/Masterarbeit", "filenames")
np.save(f"{filename}.npy",filenames) 

# Build Model

## Discriminator

In [ ]:
def apply_phaseshuffle(x, rad=2):
    b, x_len, nch = list(x.shape)

    phase = tf.random.uniform([], minval=-rad, maxval=rad + 1, dtype=tf.int32)
    pad_l = tf.maximum(phase, 0)
    pad_r = tf.maximum(-phase, 0)
    phase_start = pad_r
    x = tf.pad(x, [[0, 0], [pad_l, pad_r], [0, 0]], mode='reflect')

    x = x[:, phase_start:phase_start+x_len]
    tf.ensure_shape(x,[b, x_len, nch])

    return x

conv block für clean code später

In [ ]:
def conv_block(x, filters, kerne_size, strides, padding, n_layer,
               kernel_initializer, kernel_constraint, use_bn=True,
               use_phaseshuffle=True):
    def phaseshuffle(x): return apply_phaseshuffle(x)
    x = layers.Conv1D(filters=filters, kernel_size=kerne_size, strides=strides,
                      padding=padding,
                      kernel_initializer=kernel_initializer,
                      kernel_constraint=kernel_constraint,
                      name=f"discr_conv_layer_{n_layer}")(x)
    if use_bn:
        x = layers.BatchNormalization(name=f"discr_bn_layer_{n_layer}")(x)
    x = layers.LeakyReLU(alpha=0.2, name=f"discr_LeakyReLU_layer_{n_layer}")(x)
    if use_phaseshuffle:
        x = phaseshuffle(x)
    return x

In [ ]:
def define_discriminator(input_shape=(262144, 1)):
    def phaseshuffle(x): return apply_phaseshuffle(x)
    init = keras.initializers.RandomNormal(stddev=0.02)
    
    inputs = keras.Input(shape=input_shape, name="discriminator_inputs")
    # downsample to 65536
    x1 = layers.Conv1D(filters=64, kernel_size=25, strides=4,
                       padding="same",
                       #kernel_initializer=init,
                       name="discr_conv_layer_1")(inputs)
    x1 = layers.LeakyReLU(alpha=0.2, name="discr_LeakyReLU_layer_1")(x1)
    # downsample to 16384
    x2 = layers.Conv1D(filters=64*2, kernel_size=25, strides=4,
                       padding="same",
                       #kernel_initializer=init,
                       name="discr_conv_layer_2")(x1)
    x2 = layers.LeakyReLU(alpha=0.2, name="discr_LeakyReLU_layer_2")(x2)
    x2 = phaseshuffle(x2)
    # downsample to 4096
    x3 = layers.Conv1D(filters=64*2, kernel_size=25, strides=4,
                       padding="same",
                       #kernel_initializer=init,
                       name="discr_conv_layer_3")(x2)
    x3 = layers.LeakyReLU(alpha=0.2, name="discr_LeakyReLU_layer_3")(x3)
    x3 = phaseshuffle(x3)
    # downsample to 1024
    x4 = layers.Conv1D(filters=64*4, kernel_size=25, strides=4,
                       padding="same",
                       #kernel_initializer=init,
                       name="discr_conv_layer_4")(x3)
    x4 = layers.LeakyReLU(alpha=0.2, name="discr_LeakyReLU_layer_4")(x4)
    x4 = phaseshuffle(x4)
    # downsample to 256
    x5 = layers.Conv1D(filters=64*8, kernel_size=25, strides=4,
                       padding="same",
                       #kernel_initializer=init,
                       name="discr_conv_layer_5")(x4)
    x5 = layers.LeakyReLU(alpha=0.2, name="discr_LeakyReLU_layer_5")(x5)
    x5 = phaseshuffle(x5)
    # downsample to 64
    x6 = layers.Conv1D(filters=64*16, kernel_size=25, strides=4,
                       padding="same",
                       kernel_initializer=init,
                       name="discr_conv_layer_6")(x5)
    x6 = layers.LeakyReLU(alpha=0.2, name="discr_LeakyReLU_layer_6")(x6)
    # downsample to 16
    x7 = layers.Conv1D(filters=64*32, kernel_size=25, strides=4,
                       padding="same",
                       #kernel_initializer=init,
                       name="discr_conv_layer_7")(x6)
    x7 = layers.LeakyReLU(alpha=0.2, name="discr_LeakyReLU_layer_7")(x7)
    # classifier
    x = layers.Flatten(name="discr_flatten")(x7)
    x = layers.Dropout(0.2, name="discr_dropout")(x)
    outputs = layers.Dense(units=1, activation="linear",
                           name="discr_output")(x)
    # model
    discriminator = keras.Model(
        inputs=inputs, outputs=outputs, name="discriminator")


    return discriminator

## Generator

In [ ]:
def upsample_block():
    pass

In [ ]:
def define_generator(latent_dim):
    init = keras.initializers.RandomNormal(stddev=0.02)

    inputs = keras.Input(shape=latent_dim, name="generator_inputs")
    # filters to begin with 2048 =32*64
    n_nodes = 16*64*32
    # upsample to 16
    x = layers.Dense(units=n_nodes,
                     #kernel_initializer=init,
                     name="gener_dense_layer")(inputs)
    x = layers.Reshape(target_shape=(16, 64*32), name="gener_reshape")(x)
    x = layers.ReLU(name="gener_LeakyReLU_dense_layer")(x)

    # upsample to 64
    x1 = layers.Conv1DTranspose(filters=64*16, kernel_size=25, strides=4,
                                #kernel_initializer=init,
                                padding="same",
                                name="gener_conv_layer_1")(x)
    x1 = layers.ReLU(name="gener_LeakyReLU_layer_1")(x1)
    # upsample to 256
    x2 = layers.Conv1DTranspose(filters=64*8, kernel_size=25, strides=4,
                                #kernel_initializer=init,
                                padding="same",
                                name="gener_conv_layer_2")(x1)
    x2 = layers.ReLU(name="gener_LeakyReLU_layer_2")(x2)
    # upsample to 1024
    x3 = layers.Conv1DTranspose(filters=64*4, kernel_size=25, strides=4,
                                #kernel_initializer=init,
                                padding="same",
                                name="gener_conv_layer_3")(x2)
    x3 = layers.LeakyReLU(alpha=0.2, name="gener_LeakyReLU_layer_3")(x3)
    # upsample to 4096
    x4 = layers.Conv1DTranspose(filters=64*2, kernel_size=25, strides=4,
                                #kernel_initializer=init,
                                padding="same",
                                name="gener_conv_layer_4")(x3)
    x4 = layers.ReLU(name="gener_LeakyReLU_layer_4")(x4)
    # upsample to 16384
    x5 = layers.Conv1DTranspose(filters=64*2, kernel_size=25, strides=4,
                                #kernel_initializer=init,
                                padding="same",
                                name="gener_conv_layer_5")(x4)
    x5 = layers.ReLU(name="gener_LeakyReLU_layer_5")(x5)
    # upsample to 65536
    x6 = layers.Conv1DTranspose(filters=64, kernel_size=25, strides=4,
                                #kernel_initializer=init,
                                padding="same",
                                name="gener_conv_layer_6")(x5)
    x6 = layers.ReLU(name="gener_LeakyReLU_layer_6")(x6)
    # upsample to 262144 (~11s audio)
    x7 = layers.Conv1DTranspose(filters=1, kernel_size=25, strides=4,
                                #kernel_initializer=init,
                                padding="same",
                                name="gener_conv_layer_7")(x6)
    # activation
    outputs = keras.activations.tanh(x7)

    # model
    generator = keras.Model(inputs=inputs, outputs=outputs, name="generator")

    return generator

## GAN

https://keras.io/examples/generative/wgan_gp/

anmerkung: der hier verwendete algo weicht etwas von dem orginalen ab:
https://jonathan-hui.medium.com/gan-wasserstein-gan-wgan-gp-6a1a2aa1b490
bei dem originalen with die interpolation durch 
$\hat{x} = t*\tilde{x}+(1-t)$ mit $0<=t<=1 $
erzeugt.
Und hier durch hinufügen eines Wertes zu den real_samples

In [ ]:
class WGAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim,
                 discriminator_extra_steps=3, gp_weight=10
                 ):
        super(WGAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.d_steps = discriminator_extra_steps
        self.gp_weight = gp_weight

    def compile(self, d_optimizer, g_optimizer, d_loss_fn, g_loss_fn):
        super(WGAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.d_loss_fn = d_loss_fn
        self.g_loss_fn = g_loss_fn

    def gradient_penalty(self, batch_size, real_samples, fake_samples):

        # get inerpolated sample, so a sample which lies between the real and the fake one
        alpha = tf.random.normal(
            shape=[batch_size, 1,1], mean=0.0, stddev=1.0)
        diff = fake_samples - real_samples
        interpolated = real_samples + alpha * diff

        with tf.GradientTape() as gp_tape:
            gp_tape.watch(interpolated)
            pred = self.discriminator(interpolated, training=True)

        grads = gp_tape.gradient(pred, [interpolated])[0]
        norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=[1, 2]))
        gp = tf.reduce_mean((norm-1.0)**2)
        return gp
    
    @tf.function
    def train_step(self, real_samples):
        if isinstance(real_samples, tuple):
            real_samples = real_samples[0]
        batch_size = tf.shape(real_samples)[0]
     
        # For each batch, we are going to perform the
        # following steps as laid out in the original paper:
        # 1. Train the generator and get the generator loss
        # 2. Train the discriminator and get the discriminator loss
        # 3. Calculate the gradient penalty
        # 4. Multiply this gradient penalty with a constant weight factor
        # 5. Add the gradient penalty to the discriminator loss
        # 6. Return the generator and discriminator losses as a loss dictionary

        # Train the discriminator first. The original paper recommends training
        # the discriminator for `x` more steps (typically 5) as compared to
        # one step of the generator.
        for i in range(self.d_steps):
            # get latent vector
            random_latent_vectors = tf.random.normal(
                shape=(batch_size, self.latent_dim)
            )
            with tf.GradientTape() as tape:
                # generate fake sample form latent vector
                fake_samples = self.generator(
                    random_latent_vectors, training=True)
                # get the logits for the fake samples
                fake_logits = self.discriminator(fake_samples, training=True)
                # get the logits for the reals samples
                real_logits = self.discriminator(real_samples, training=True)
                # calculate the discriminator loss using fake and real sample logits
                d_cost = self.d_loss_fn(real_sample=real_logits,
                                        fake_sample=fake_logits)
                # calclate the gradient penalty
                gp = self.gradient_penalty(
                    batch_size, real_samples, fake_samples)
                # add the gradient penalty to the original discriminator loss
                d_loss = d_cost + gp * self.gp_weight

            # get the gradient w.r.t the discriminator loss
            d_gradient = tape.gradient(
                d_loss, self.discriminator.trainable_variables)
            # update the weights of the discriminator using the discrminator optimizer
            self.d_optimizer.apply_gradients(
                zip(d_gradient, self.discriminator.trainable_variables)
            )  
        # train the generator
        # get the latent vector
        random_latent_vectors = tf.random.normal(shape=(batch_size,self.latent_dim))
        with tf.GradientTape() as tape:
            #generate fake images using the generator
            generated_samples = self.generator(random_latent_vectors,training=True)
            # get the discriminator logits for fake images
            gen_sample_logits = self.discriminator(generated_samples,training=True)
            # calculate the generator loss
            g_loss = self.g_loss_fn(gen_sample_logits)
        # get the gradients w.r.t. the generator loss
        gen_gradient = tape.gradient(g_loss, self.generator.trainable_variables)
        # update the weights of the generator using the genrator optimizer
        self.g_optimizer.apply_gradients(
            zip(gen_gradient, self.generator.trainable_variables)
        )
        return {"d_loss":d_loss, "g_loss":g_loss}

# Preprocessing / Data Loading

In [ ]:
signals = np.load("./waveform_data.npy",mmap_mode="r",allow_pickle=True)
signals = np.expand_dims(signals, axis=-1)

In [ ]:
class BatchGenerator(tf.keras.utils.Sequence) :
  
    def __init__(self, filenames, batch_size) :
        self.filenames = filenames
        self.batch_size = batch_size
    
    
    def __len__(self) :
        return (np.ceil(len(self.filenames) / float(self.batch_size))).astype(np.int)
  
  
    def __getitem__(self, idx) :
        batch_x = self.filenames[idx * self.batch_size : (idx+1) * self.batch_size]
        
        sound_files = np.array([np.load(f"{FILES_DIR_SAVE}/{file_name}",allow_pickle=True)
                         for file_name in batch_x])
        sound_files = np.expand_dims(sound_files, axis=-1)
        return sound_files

# Utils

In [ ]:
def get_random_vector(n_samples,latent_dim):
    return tf.random.uniform(shape=(n_samples,latent_dim),minval=-1, maxval=1)

# Train

In [ ]:
import soundfile as sf
from matplotlib import pyplot as plt

global g_loss_history, d_loss_history
g_loss_history, d_loss_history = [],[]

class GANMonitor(keras.callbacks.Callback):
    def __init__(self,latent_dim,sample_rate):
        self.latent_dim = latent_dim
        self.sample_rate = sample_rate
        
    def on_epoch_end(self,epoch,logs=None):
        if epoch % 5 == 0:
            random_latent_vector = tf.random.normal(shape=(1,self.latent_dim))
            generated_sample = self.model.generator(random_latent_vector)
            generated_sample = generated_sample.numpy().reshape((262144,))
            sf.write(f"./song_after_epoch_{epoch}.wav", generated_sample, self.sample_rate)

            model_name = f"model_after_{epoch}_epochs.h5"
            self.model.generator.save(model_name)
            
            plt.plot(d_loss_history, label='d_loss')
            plt.plot(g_loss_history, label='g_loss')
            plt.legend()
            plt.savefig(f'loss_histroy_after_epoch_{epoch}.png')
            plt.close()
        
    def on_train_batch_end(self,batch,logs=None):
        d_loss_history.append(logs["d_loss"])
        g_loss_history.append(logs["g_loss"])
        
    
        

In [ ]:
import time

# set number of epochs
EPOCHS = 1
# set latent dim
latent_dim = 500
# set batch_size
BATCH_SIZE = 4

# instantiate model
g_model = define_generator(latent_dim)
d_model = define_discriminator()

# istantiate optimizers
generator_optimizer = keras.optimizers.Adam(learning_rate=0.0002,
                                            beta_1=0.5,
                                            beta_2=0.9
                                            )
discriminator_optimizer = keras.optimizers.Adam(learning_rate=0.0002,
                                               beta_1=0.5,
                                               beta_2=0.9)
# define losses
# gp is added later to discriminator loss


def discriminator_loss(real_sample, fake_sample):
    real_loss = tf.reduce_mean(real_sample)
    fake_loss = tf.reduce_mean(fake_sample)
    return fake_loss - real_loss


def generator_loss(fake_sample):
    return -tf.reduce_mean(fake_sample)



# instantiate the WGAN model
wgan = WGAN(
    discriminator=d_model,
    generator=g_model,
    latent_dim=latent_dim,
    discriminator_extra_steps=5
)
wgan.compile(
    d_optimizer=discriminator_optimizer,
    g_optimizer=generator_optimizer,
    g_loss_fn=generator_loss,
    d_loss_fn=discriminator_loss
)

callbacks = [GANMonitor(latent_dim=latent_dim,sample_rate=SAMPLE_RATE)]

training_data = BatchGenerator(np.load(f"{FILENAME_DIR}/filenames.npy",allow_pickle=True),BATCH_SIZE)



In [ ]:
start = time.time()
wgan.fit(training_data, batch_size=BATCH_SIZE,epochs=EPOCHS,callbacks=callbacks,verbose=1)
end = time.time()
process_time = end - start
print(f'Process time for training: {round(process_time,3)}')

# Generate Data

In [ ]:
# example of loading the generator model and generating images
from tensorflow.keras.models import load_model
from matplotlib import pyplot as plt



# load model
model = load_model('./models/WGAN-GP_24_09/generator_after_20_epochs.h5')


In [ ]:
model.summary()

In [ ]:
# generate images
latent_points = get_random_vector(1,latent_dim=256)
# generate images
X = model.predict(latent_points)
# plot the result
#show_plot(X, 10)

In [ ]:
sound_file = X[0].reshape((262144,))

# Evaluate

In [ ]:
from matplotlib import pyplot as plt
import librosa.display

In [ ]:
np.array(model.get_weights())[6]

## generated soundfile

In [ ]:
signal1

In [ ]:
SAMPLE_RATE = 22050
import soundfile as sf
sf.write("./test.wav", signal, SAMPLE_RATE)

In [ ]:
wave = sound_file[0].reshape((262144,))

In [ ]:
plt.figure(figsize=(15,17))
plt.subplot(3,1,1)
librosa.display.waveplot(signal,alpha = 0.5)